In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

Using TensorFlow backend.


In [2]:
#Functions to compute Precision Recall and F1 Score
import keras.backend as K
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1_score(y_true, y_pred):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f1_score = 2 * (p * r) / (p + r + K.epsilon())
    return f1_score

In [3]:
#Read images from relvant os/G Drive directories
#train_dir=r'/content/drive/My Drive/42.ExtLab02Jun2019/train'
#test_dir=r'/content/drive/My Drive/42.ExtLab02Jun2019/test'
train_dir=r'C:\Users\sinamy\Documents\GreatLearning\42.ExtLab02Jun2019\train'
test_dir=r'C:\Users\sinamy\Documents\GreatLearning\42.ExtLab02Jun2019\test'
datagen = ImageDataGenerator(rescale = 1./255,
                             featurewise_center=False,  # set input mean to 0 over the dataset
                             samplewise_center=False,  # set each sample mean to 0
                             featurewise_std_normalization=False,  # divide inputs by std of the dataset
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening
                             rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
                             width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=True,
                             validation_split=0.2)
train_gen = datagen.flow_from_directory(directory=train_dir,
                                        target_size = (256, 256),
                                        class_mode='categorical',
                                        subset='training')
val_gen = datagen.flow_from_directory(directory=train_dir,
                                        target_size = (256, 256),
                                        class_mode='categorical',
                                        subset='validation')

Found 3799 images belonging to 12 classes.
Found 945 images belonging to 12 classes.


In [4]:
#Sample CNN Architecture with One Epoch
BATCH_SIZE = 32
EPOCH = 1
# neural network model
model = Sequential()
model.add(Convolution2D(64, (5,5), input_shape = (256, 256, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Convolution2D(32, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Convolution2D(32, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Convolution2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy',f1_score])

model.fit_generator(train_gen,
                    steps_per_epoch = 4750//BATCH_SIZE,
                    epochs = EPOCH)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
148/148 [==============================] - 921s 6s/step - loss: 2.6221 - acc: 0.1177 - f1_score: 0.0044


In [5]:
#Use of Transfer Learning with VGG19 Architecture with additional dense layesr to learn more complex functions and classify for better result
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras import optimizers
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Create the base model of VGG19
base_model = VGG19(weights='imagenet', include_top=False, input_shape = (256, 256, 3), classes = 12)
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layer 1
x = Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x = Dropout(0.5)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(12,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
BATCH_SIZE = 32
EPOCH = 50
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics = ['accuracy',f1_score])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model.fit_generator(train_gen,
                    steps_per_epoch = 3803 // BATCH_SIZE,
                    validation_data = val_gen,
                    validation_steps = 947 // BATCH_SIZE,
                    epochs = EPOCH,
                    callbacks = [checkpoint, early])

In [6]:
#Use of Transfer Learning with InceptionResNetV2 Architecture with additional dense layesr to learn more complex functions and classify for better result
from keras.applications import InceptionResNetV2
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras import optimizers
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Create the base model of VGG19
base_model_1 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = (256, 256, 3), classes = 12)
x=base_model_1.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x = Dropout(0.5)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(12,activation='softmax')(x) #final layer with softmax activation

model_1=Model(inputs=base_model_1.input,outputs=preds)

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 13s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model_1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [8]:
BATCH_SIZE = 32
EPOCH = 50
model_1.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics = ['accuracy',f1_score])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("Inception_resnet.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_1.fit_generator(train_gen,
                    steps_per_epoch = 3803 // BATCH_SIZE,
                    validation_data = val_gen,
                    validation_steps = 947 // BATCH_SIZE,
                    epochs = EPOCH,
                    callbacks = [checkpoint, early])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
118/118 [==============================] - 2970s 25s/step - loss: 2.5301 - acc: 0.0951 - val_loss: 2.4430 - val_acc: 0.1530

Epoch 00001: val_acc improved from -inf to 0.15302, saving model to Inception_resnet.h5
Epoch 2/50
118/118 [==============================] - 141s 1s/step - loss: 2.4476 - acc: 0.1268 - val_loss: 2.3999 - val_acc: 0.1803

Epoch 00002: val_acc improved from 0.15302 to 0.18033, saving model to Inception_resnet.h5
Epoch 3/50
118/118 [==============================] - 142s 1s/step - loss: 2.4181 - acc: 0.1468 - val_loss: 2.3638 - val_acc: 0.1847

Epoch 00003: val_acc improved from 0.18033 to 0.18470, saving model to Inception_resnet.h5
Epoch 4/50
118/118 [==============================] - 140s 1s/step - loss: 2.3844 - acc: 0.1654 - val_loss: 2.3293 - val_acc: 0.1836

Epoch 00004: val_acc did not improve from 0.18470
Epoch 5/50
118/118 [==============================] - 141s 1s/step - loss: 2.3705 - acc: 0.162

In [10]:
#Use of Transfer Learning with ResNet50 Architecture with additional dense layesr to learn more complex functions and classify for better result
from keras.applications import ResNet50
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras import optimizers
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Create the base model of VGG19
base_model_2 = ResNet50(weights='imagenet', include_top=False, input_shape = (256, 256, 3), classes = 12)
x=base_model_2.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x = Dropout(0.5)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(12,activation='softmax')(x) #final layer with softmax activation

model_2=Model(inputs=base_model_2.input,outputs=preds)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [11]:
model_2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [12]:
BATCH_SIZE = 32
EPOCH = 50
model_2.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics = ['accuracy',f1_score])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("resnet50.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_2.fit_generator(train_gen,
                    steps_per_epoch = 3803 // BATCH_SIZE,
                    validation_data = val_gen,
                    validation_steps = 947 // BATCH_SIZE,
                    epochs = EPOCH,
                    callbacks = [checkpoint, early])

Epoch 1/50
118/118 [==============================] - 144s 1s/step - loss: 2.5976 - acc: 0.0991 - val_loss: 2.3375 - val_acc: 0.1760

Epoch 00001: val_acc improved from -inf to 0.17596, saving model to resnet50.h5
Epoch 2/50
118/118 [==============================] - 118s 996ms/step - loss: 2.4499 - acc: 0.1362 - val_loss: 2.2770 - val_acc: 0.2328

Epoch 00002: val_acc improved from 0.17596 to 0.23279, saving model to resnet50.h5
Epoch 3/50
118/118 [==============================] - 123s 1s/step - loss: 2.3655 - acc: 0.1781 - val_loss: 2.1410 - val_acc: 0.4098

Epoch 00003: val_acc improved from 0.23279 to 0.40984, saving model to resnet50.h5
Epoch 4/50
118/118 [==============================] - 123s 1s/step - loss: 2.2598 - acc: 0.2239 - val_loss: 2.0732 - val_acc: 0.4098

Epoch 00004: val_acc did not improve from 0.40984
Epoch 5/50
118/118 [==============================] - 123s 1s/step - loss: 2.1646 - acc: 0.2768 - val_loss: 1.8868 - val_acc: 0.4995

Epoch 00005: val_acc improved f

In [44]:
classes = train_gen.class_indices    
classes

{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

In [9]:
from keras.models import load_model
#vgg = load_model("vgg16_1.h5")
#vgg = load_model("Inception_resnet.h5")
vgg = load_model("resnet50.h5")
vgg.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
              metrics = ['accuracy',f1_score])

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(256, 256),
                                             color_mode="rgb",
                                             batch_size=1,
                                             class_mode=None,
                                             shuffle=False,
                                             seed=42)

Found 794 images belonging to 1 classes.


In [10]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=vgg.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

794/794 [==============================] - 383s 483ms/step


In [11]:
predicted_class_indices=np.argmax(pred,axis=1)

In [15]:
labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [19]:
filenames=test_generator.filenames
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("submission.csv",index=False)

In [ ]:
#Public Score on Kaggle : 0.93576 with ResNet50 CNN architecture